In [1]:
#import libraries
import pandas as pd
from datetime import datetime
from datetime import timedelta

pd.options.display.max_columns = None # no limits to columns display

In [2]:
#import datasets
path = ""
loans_lenders = pd.read_csv("loans_lenders.csv", nrows=100000)
lenders = pd.read_csv("lenders.csv")
loans = pd.read_csv("loans.csv", nrows=100000)

In [3]:
# 1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lenders
loans_lenders = pd.concat([pd.Series(row['loan_id'], row['lenders'].split(',')) for _,row in loans_lenders.iterrows()]).reset_index()
loans_lenders.columns = ['permanent_name', 'loan_id']
loans_lenders.head()

,permanent_name,loan_id
0,muc888,483693
1,sam4326,483693
2,camaran3922,483693
3,lachheb1865,483693
4,rebecca3499,483693


In [4]:
# 2) For each loan, add a column duration corresponding to the number of days between 
#the disburse time and the planned expiration time. 
def time_delta(y,x): 
    end = pd.to_datetime(y)
    start = pd.to_datetime(x)
    delta = end-start
    return delta

loans["duration"] =  time_delta(loans.planned_expiration_time, loans.disburse_time)
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02


In [5]:
# 3) find the "lenders" that have funded at least twice.
not_unique_lenders = loans_lenders.groupby("permanent_name").size()
not_unique_lenders = pd.DataFrame(not_unique_lenders[not_unique_lenders  > 1])
not_unique_lenders.columns = ["number_of_loans"]
not_unique_lenders.head()

,number_of_loans
permanent_name,
000,3
00000,5
0002,3
0101craign0101,4
0326lsw,3


In [14]:
#4) For each country, compute how many loans have involved that country as borrowers.
#5) For each country, compute the overall amount of money borrowed.
aggregate_state = pd.concat([
    loans.groupby(['country_code']).count()['loan_id'],
    loans.groupby(['country_code']).sum()['funded_amount']], axis = 1)
aggregate_state.columns = ('count_loans', 'amount_loans')
aggregate_state.head()

,count_loans,amount_loans
country_code,,
AF,98,86125.0
AL,241,314775.0
AM,971,1429575.0
AZ,661,1043300.0
BA,29,20475.0


In [15]:
# 6) Like the previous point, but expressed as a percentage of the overall amount lent.
merged = loans.merge(loans_lenders,on='loan_id').merge(lenders,on='permanent_name')
lent = merged.groupby(['country_code_y'])['funded_amount'].sum()
aggregate_state = pd.concat([aggregate_state,lent],axis=1,ignore_index=True, sort=False).fillna(0)
aggregate_state.columns = ('count_loans', 'amount_loans', 'lent_amount')
aggregate_state["percentage"] = aggregate_state["lent_amount"] / aggregate_state["amount_loans"] *100
aggregate_state.head()

,count_loans,amount_loans,lent_amount,percentage
AF,98.0,86125.0,700.0,0.812772
AL,241.0,314775.0,1400.0,0.444762
AM,971.0,1429575.0,0.0,0.000000
AZ,661.0,1043300.0,475.0,0.045529
BA,29.0,20475.0,1725.0,8.424908


In [16]:
# 7) Like the three previous points, but split for each year (with respect to disburse time).
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans['year'] = loans['disburse_time'].dt.year
aggregate_state_years = pd.concat([
    loans.groupby(['country_code', 'year']).count()['loan_id'],
    loans.groupby(['country_code', 'year']).sum()['funded_amount']], axis = 1)
aggregate_state_years.columns = ('count_loans', 'amount_loans')
aggregate_state_years.head()

count_loans  amount_loans
country_code year                             
AF           2007.0           10        5750.0
             2008.0            6        5875.0
             2009.0           51       44950.0
             2010.0           25       23450.0
             2011.0            6        6100.0

In [17]:
# 8) For each lender, compute the overall amount of money lent.
merged['single_loan_amount']= merged['funded_amount'] / merged['num_lenders_total']
lender_amount = merged.groupby(['permanent_name']).sum()['single_loan_amount']
lender_amount.head()

permanent_name
0416503                37.500000
0819212                29.411765
0983056                26.470588
0li                    56.250000
100ofhumanity1199    2244.972649
Name: single_loan_amount, dtype: float64

In [18]:
# 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed.
aggregate_state["difference"] = aggregate_state["lent_amount"] - aggregate_state["amount_loans"]
aggregate_state.head()

,count_loans,amount_loans,lent_amount,percentage,difference
AF,98.0,86125.0,700.0,0.812772,-85425.0
AL,241.0,314775.0,1400.0,0.444762,-313375.0
AM,971.0,1429575.0,0.0,0.000000,-1429575.0
AZ,661.0,1043300.0,475.0,0.045529,-1042825.0
BA,29.0,20475.0,1725.0,8.424908,-18750.0


In [19]:
# 10) Which country has the highest ratio between the difference computed at the previous point and the population?
country_stats =pd.read_csv("country_stats.csv")
aggregate_state.reset_index(inplace = True)
aggregate_state=aggregate_state.rename(columns = {'index':'country_code'})
aggregate_state_extended = aggregate_state.merge(country_stats, on = "country_code")
aggregate_state_extended['ratio'] = aggregate_state_extended['difference'] / aggregate_state_extended['population']
aggregate_state_extended.iloc[aggregate_state_extended['ratio'].idxmax()]

country_code                                  NO
count_loans                                    0
amount_loans                                   0
lent_amount                               509325
percentage                                   inf
difference                                509325
country_name                              Norway
country_code3                                NOR
continent                                 Europe
region                           Northern Europe
population                               5305383
population_below_poverty_line                NaN
hdi                                     0.949423
life_expectancy                           81.711
expected_years_of_schooling              17.6719
mean_years_of_schooling                  12.7464
gni                                      67614.4
kiva_country_name                         Norway
ratio                                  0.0960016
Name: 125, dtype: object

In [20]:
#11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?
aggregate_state_extended['ratio_not_poverty'] = aggregate_state_extended['difference'] / (aggregate_state_extended['population'] * (100 - aggregate_state_extended['population_below_poverty_line'] ))
aggregate_state_extended.iloc[aggregate_state_extended['ratio_not_poverty'].idxmax()]

country_code                                   CA
count_loans                                     0
amount_loans                                    0
lent_amount                           3.22205e+06
percentage                                    inf
difference                            3.22205e+06
country_name                               Canada
country_code3                                 CAN
continent                                Americas
region                           Northern America
population                               36624199
population_below_poverty_line                 9.4
hdi                                      0.920284
life_expectancy                            82.224
expected_years_of_schooling                16.325
mean_years_of_schooling                   13.1051
gni                                       42581.9
kiva_country_name                          Canada
ratio                                    0.087976
ratio_not_poverty                     0.000971037


In [16]:
#12
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
def delta_date(start, stop):
    delta = stop - start
    
    if delta.days < 0 or pd.isnull(start) or pd.isnull(stop): #check error
        return {}
    years = {}
    for i in range(delta.days + 1):
        day = start + timedelta(days=i)
        if not day.year in years:
            years[day.year] = 1;
        else:
            years[day.year] += 1
    return years
group_years = pd.DataFrame(columns = ["year","amount"])
group_years.set_index('year', inplace=True)
for _, row in loans.iterrows():
    years = delta_date(row["disburse_time"], row["planned_expiration_time"])
    if (row["planned_expiration_time"] - row["disburse_time"]).days == 0:
        continue
    per_day = row["loan_amount"] / (row["planned_expiration_time"] - row["disburse_time"]).days 
    for key, value in years.items():
        if key not in group_years.index:
            group_years.loc[key] = per_day * value
        else:
           group_years["amount"][key] += per_day * value
group_years

,amount
year,
2013,1.421598e+07
2014,1.474127e+07
2015,8.377948e+06
2012,9.098303e+06
2016,9.028823e+06
2017,1.167408e+07
2018,2.353648e+05
2011,2.218274e+04


In [21]:
#2.1 For each value of repayment_interval, add a new column to the lenders dataframe 
#that contains the total amount of money corresponding to loans in such state
merged = pd.merge(merged, merged.pivot(index = 'loan_id', columns='repayment_interval', values='single_loan_amount').fillna(0), on='loan_id')
lenders = pd.merge(lenders, merged[['bullet', 'irregular', 'monthly', 'weekly', 'permanent_name']], on='permanent_name', how='inner')
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,bullet,irregular,monthly,weekly
0,vandhana3936,Vandhana,NaN,NaN,NaN,1461316675,NaN,NaN,7.0,NaN,0,0.000000,30.769231,0.000000,0.0
1,torsten6216,Torsten,NaN,NaN,NaN,1461326898,NaN,NaN,3.0,NaN,0,0.000000,0.000000,92.857143,0.0
2,brigitte5423,Brigitte,NaN,NaN,NaN,1461324009,NaN,NaN,3.0,NaN,0,34.166667,0.000000,0.000000,0.0
3,alex38245581,Alex,Bavaria,NaN,DE,1461438451,NaN,NaN,438.0,Sharon,0,0.000000,0.000000,38.888889,0.0
4,kristoffer2583,Kristoffer,NaN,NaN,NaN,1461560154,NaN,NaN,3.0,NaN,0,40.625000,0.000000,0.000000,0.0


In [22]:
#2.2 What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?
occupation = pd.DataFrame(merged.groupby(['activity_name']).sum()['single_loan_amount'])
occupation.reset_index(level=0, inplace=True)
max_occupation = occupation['single_loan_amount'].idxmax()
occupation.loc[max_occupation]

activity_name         General Store
single_loan_amount           241012
Name: 70, dtype: object

In [25]:
#2.3 Cluster the loans according to the year-month of disburse time.
clusters = [x for _, x in loans.groupby([(loans['disburse_time'].dt.year) , (loans['disburse_time'].dt.month)])]
clusters[0]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration,year
85222,89,Rose,English,<i>\r\nThe following description was written b...,NaN,500.0,500.0,funded,Food Production/Sales,Food,Buying and reselling of produce,UG,Uganda,Tororo,not shared,NaN,USD,1.0,2006-04-16 07:10:50.000 +0000,NaN,2005-04-14 05:27:55+00:00,2005-03-31 06:27:55.000 +0000,6.0,6,28,14,NaN,female,true,bullet,field_partner,NaT,2005.0
85369,241,Angelina Lopez,English,My name is Angelina LÃ³pez and I sell accessor...,NaN,350.0,350.0,funded,Tailoring,Services,"Buying cloth, zippers, buttons and lace",HN,Honduras,Danli,not shared,NaN,USD,5.0,2006-04-16 07:10:50.000 +0000,NaN,2005-04-14 05:27:55+00:00,2005-03-31 06:27:55.000 +0000,12.0,4,4,2,NaN,female,true,bullet,field_partner,NaT,2005.0
89113,143,NaN,NaN,NaN,NaN,500.0,500.0,funded,Bricks,Construction,NaN,KE,Kenya,NaN,not shared,NaN,USD,1.0,2006-04-16 07:10:50.000 +0000,NaN,2005-04-14 05:27:55+00:00,2005-03-31 06:27:55.000 +0000,6.0,6,8,4,NaN,NaN,NaN,bullet,field_partner,NaT,2005.0
99222,231,Blanca Nieves Arguet,English,My family and I serve food for a living. I ha...,NaN,650.0,650.0,funded,Food Production/Sales,Food,"Buying rice, lard, beans and dairy products.",HN,Honduras,Danli,not shared,NaN,USD,5.0,2006-04-16 07:10:50.000 +0000,NaN,2005-04-14 05:27:55+00:00,2005-03-31 06:27:55.000 +0000,15.0,12,4,2,NaN,female,true,bullet,field_partner,NaT,2005.0


In [26]:
#2.4 For each country, compute its overall GNI, by multiplying the per capita GNI with its population.
country_stats = pd.read_csv('country_stats.csv')
country_stats['overall_GNI'] = country_stats['gni'] * country_stats['population']
country_stats.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,overall_GNI
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,7.584413e+12
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,1.038975e+12
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,2.116096e+12
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,9.912214e+11
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh,5.502424e+11
